In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [4]:
image = cv2.imread('test.jpg')
corners, ids, rejected = cv2.aruco.detectMarkers(image, arucoDict7,parameters=arucoParams)
print(corners)

image = process_frame(image)
cv2.imshow("test", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

[array([[[852., 449.],
        [910., 449.],
        [909., 511.],
        [851., 511.]]], dtype=float32), array([[[466., 448.],
        [533., 448.],
        [531., 514.],
        [465., 514.]]], dtype=float32), array([[[858., 219.],
        [917., 221.],
        [915., 283.],
        [856., 282.]]], dtype=float32), array([[[470., 203.],
        [537., 206.],
        [536., 272.],
        [469., 270.]]], dtype=float32)]


In [2]:
arucoDict7 = cv2.aruco.Dictionary_get(cv2.aruco.DICT_7X7_50)
arucoDict6 = cv2.aruco.Dictionary_get(cv2.aruco.DICT_6X6_50)
arucoParams = cv2.aruco.DetectorParameters_create()

In [21]:
image = cv2.imread('test.jpg')
corners = np.array(corners)
corners = corners.astype('int')
cv2.circle(image, tuple(corners[3][0][0].astype('int')), 3, (0, 255, 0), 3)
cv2.circle(image, tuple(corners[2][0][1].astype('int')), 3, (0, 255, 0), 3)
cv2.circle(image, tuple(corners[1][0][3].astype('int')), 3, (0, 255, 0), 3)
cv2.circle(image, tuple(corners[0][0][2].astype('int')), 3, (0, 255, 0), 3)

cv2.imshow("test", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [112]:
tl = tuple(corners[3][0][0].astype('int'))
tr = tuple(corners[2][0][1].astype('int'))
bl = tuple(corners[1][0][3].astype('int'))
br = tuple(corners[0][0][2].astype('int'))

In [113]:
width = 1500
height = 1000
pts_src = np.array([tl, tr, bl, br])
pts_dst = np.array([[0,0], [width,0], [0,height], [width,height]])
h, status = cv2.findHomography(pts_src, pts_dst)
img_warp = cv2.warpPerspective(image, h, (width,height))
cv2.imshow("test", img_warp)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
image = cv2.imread('test2.jpg')
image = process_frame(image)
cv2.imshow("test", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[ 1.77972918e+00 -1.01079953e-01 -5.96404214e+02]
 [ 9.07867028e-02  1.85785909e+00 -3.87656679e+02]
 [-1.09012737e-04 -5.79933802e-06  1.00000000e+00]]
[[[605.75 338.  ]]]
[440 276]


In [6]:
# run video
vid = cv2.VideoCapture(0, cv2.CAP_DSHOW)
vid.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
  
while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = vid.read()
  
    frame = process_frame(frame)

    # Display the resulting frame
    cv2.imshow('frame', frame)
      
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
vid.release()
cv2.destroyAllWindows()

FileNotFoundError: [Errno 2] No such file or directory: 'calibration/calibration_matrix_webcam.npy'

In [5]:
def process_frame(frame):
    mtx = np.load('calibration/calibration_matrix_webcam.npy')
    dist = np.load('calibration/distortion_coefficients_webcam.npy')
    h, w = frame.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
    frame = cv2.undistort(frame, mtx, dist, None, newcameramtx)
    
    corners, ids, rejected = cv2.aruco.detectMarkers(frame, arucoDict7,parameters=arucoParams)
    corners_v, ids_v, rejected_v = cv2.aruco.detectMarkers(frame, arucoDict6,parameters=arucoParams)
    
    
    if len(corners) < 4 or len(corners_v) < 1:
        return frame
    tl = np.array([0,0], dtype=np.float32)
    tr = np.array([0,0], dtype=np.float32)
    bl = np.array([0,0], dtype=np.float32)
    br = np.array([0,0], dtype=np.float32)
    
    v1 = np.array([0,0], dtype=np.float32)
    
    tl_index = np.where(ids == 20)[0][0]
    tr_index = np.where(ids == 21)[0][0]
    bl_index = np.where(ids == 22)[0][0]
    br_index = np.where(ids == 23)[0][0]
    
    v1_index = np.where(ids_v == 24)[0][0]

    for i in range(4):
        tl += corners[tl_index][0][i]/4
        tr += corners[tr_index][0][i]/4
        bl += corners[bl_index][0][i]/4
        br += corners[br_index][0][i]/4
        
        v1 += corners_v[v1_index][0][i]/4
    
#     cv2.circle(frame, tuple(tl.astype('int')), 3, (0, 255, 0), 3)
#     cv2.circle(frame, tuple(tr.astype('int')), 3, (0, 255, 0), 3)
#     cv2.circle(frame, tuple(bl.astype('int')), 3, (0, 255, 0), 3)
#     cv2.circle(frame, tuple(br.astype('int')), 3, (0, 255, 0), 3)
#     return frame

    
    width = 1280
    height = 720
    offset = 0
    pts_src = np.array([tl, tr, bl, br])
    pts_dst = np.array([[offset,offset], [width-offset,offset], [offset,height-offset], [width-offset,height-offset]])
    h, status = cv2.findHomography(pts_src, pts_dst)
    img_warp = cv2.warpPerspective(frame, h, (width,height))
    print(h)
    
    v1 = v1.reshape(-1,1,2)
    print(v1)
    v1 = cv2.perspectiveTransform(v1, h)[0][0].astype(int)
    v1 = v1 - [40, 40]
    print(v1)
    
    v1_color = color=(255,255,0)
    img_warp = cv2.putText(img=img_warp, text=f'({v1[0]}, {v1[1]})', org=(v1[0], v1[1]), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=v1_color, thickness=3)
    img_warp = cv2.circle(img_warp, (v1[0],v1[1]), radius=3, color=v1_color, thickness=3)
    return img_warp